In [49]:
import time
import numpy as np
from datetime import datetime
from calendar import timegm
from pymongo import MongoClient, ASCENDING, DESCENDING, ReturnDocument
import pyvisa
import serial
from pprint import pprint

## Connect to Database

In [55]:
def dbConnect():
    client = MongoClient('mongodb://localhost/')
    db = client.data
    collection = db.adr2datas
    db.collection_names()
    infocollection = db.adr2controls
    return collection, infocollection
collection,infocollection = dbConnect()

##Connect to Instruments

In [40]:
def connectInstruments():
    rm = pyvisa.ResourceManager()
    rm.list_resources()
    inst1 = rm.open_resource('GPIB0::1::INSTR', read_termination = '\x00')
    inst2 = rm.open_resource('GPIB0::2::INSTR', read_termination = '\x00')
    inst3 = rm.open_resource('GPIB0::6::INSTR', read_termination = '\x00')
    return [inst1,inst2,inst3]

##Create Information and Data Entries

In [58]:
def get_info():
    post = {
        "timeStamp": timegm(datetime.now().timetuple()),
        "fridgeStatus": "Warm",
        "currentJob": "None",
        "switchState": "Closed",
        "currentLimit": 9.2,
        "maxVoltage": 380,
        "voltageStep": 1,
        "command": "None"
    }
    return post


In [59]:
infocollection.insert(get_info())

ObjectId('55f597a86545eb56047e1f82')

In [61]:
x = infocollection.find_one()
x

{u'_id': ObjectId('55f5979f6545eb56047e1f81'),
 u'command': u'None',
 u'currentJob': u'None',
 u'currentLimit': 9.2,
 u'fridgeStatus': u'Warm',
 u'maxVoltage': 380,
 u'switchState': u'Closed',
 u'timeStamp': 1442144095,
 u'voltageStep': 1}

In [41]:
def get_data(inst):
    post = {
        "timeStamp": timegm(datetime.now().timetuple()),
        "baseTemp": float(inst[1].ask('KRDG? B')),
        "oneKTemp": float(inst[0].ask('KRDG? C')),
        "threeKTemp": float(inst[0].ask('KRDG? A')),
        "sixtyKTemp": float(inst[0].ask('KRDG? B')),
        "magnetVoltage": float(inst[0].ask('SRDG? D')),
        "psVoltage": float(inst[2].ask('MEAS:VOLT?')),
        "psCurrent": 0,
        "currentJob": "None",
        "percentComplete": 0,
        "switchState": "Closed"
    }
    return post
get_data(connectInstruments())

C:\Anaconda\lib\site-packages\pyvisa\resources\messagebased.py:387: UserWarning: read string doesn't end with termination characters
  return self.read()


{'baseTemp': 0.0,
 'currentJob': 'None',
 'magnetVoltage': -0.0005,
 'oneKTemp': 183.555,
 'percentComplete': 0,
 'psCurrent': 0,
 'psVoltage': -0.00941257,
 'sixtyKTemp': 254.054,
 'switchState': 'Closed',
 'threeKTemp': 199.048,
 'timeStamp': 1442089425}

## Setup Log File

In [43]:
logFileName = 'ADR2Log.txt'
inst = connectInstruments()

In [78]:
# Open Log File
logFile = open(logFileName, 'a')

# Read Data
try:
    data = get_data(inst)

# Log errors if they occur and reconnect
except UnicodeDecodeError:
    string = "Read error at " + datetime.now().strftime('%H:%M:%S') + '\n'
    print(string)
    logFile.write(string)
    inst = connectInstruments()

# Insert Data into Database
else:
    if (data['sixtyKTemp'] <= 300):
        try:
            collection.insert(data)
        except:
            print("Database error at " + datetime.now().strftime('%H:%M:%S'))


# Get information from Database

info = infocollection.find().next()
# print('Fridge Status:')
# pprint(info)

# Control

# Change Fridge Status

# Change Job
print info['command']
if info['command'] == 'Magup':
    if info['currentJob'] != 'Magup':
        info = infocollection.find_one_and_update({},{'$set': {'currentJob': 'Magup'}})
        logString = "Start Magup at " + datetime.now().strftime('%H:%M:%S')
        print(logString)
        logFile.writelines([logString, '\n'])

elif info['command'] == 'Magdown':
    if info['currentJob'] != 'Magdown':
        info = infocollection.find_one_and_update({},{'$set': {'currentJob': 'Magdown'}})
        logString = "Start Magdown at " + datetime.now().strftime('%H:%M:%S')
        print(logString)
        logFile.writelines([logString, '\n'])

elif info['command'] == 'Soak':
    if info['currentJob'] != 'Soak':
        info = infocollection.find_one_and_update({},{'$set': {'currentJob': 'Soak'}})
        logString = "Start Soak at " + datetime.now().strftime('%H:%M:%S')
        print(logString)
        logFile.writelines([logString, '\n'])
        
else:
    print('No Command')

# Control Magnet
job = info['currentJob']

if job == 'Magup':
    # Increase Magnet Voltage
    print('magup')
elif job == 'Magdown':
    # Decrease Magnet Voltage
    print('magdown')
elif job == 'Soak':
    # Soak
    print('soak')
else:
    print('No Job')

# Close Log File
logFile.close()

# Wait
# time.sleep(5)

Fridge Status:
{u'_id': ObjectId('55f5979f6545eb56047e1f81'),
 u'command': u'Magdown',
 u'currentJob': u'Magdown',
 u'currentLimit': 9.2,
 u'fridgeStatus': u'Warm',
 u'maxVoltage': 380,
 u'switchState': u'Closed',
 u'timeStamp': 1442144095,
 u'voltageStep': 1}
Magdown
magdown


In [73]:
'Magdown' == info['command']

True

In [ ]:
while True:
    # Open Log File
    logFile = open(logFileName, 'a')

    # Read Data
    try:
        data = get_data(inst)

    # Log errors if they occur and reconnect
    except UnicodeDecodeError:
        string = "Read error at " + datetime.now().strftime('%H:%M:%S') + '\n'
        print(string)
        logFile.write(string)
        inst = connectInstruments()

    # Insert Data into Database
    else:
        if (data['sixtyKTemp'] <= 300):
            try:
                collection.insert(data)
            except:
                print("Database error at " + datetime.now().strftime('%H:%M:%S'))


    # Get information from Database

    info = infocollection.find().next()
    # print('Fridge Status:')
    # pprint(info)

    # Control

    # Change Fridge Status

    # Change Job
#     print info['command']
    if info['command'] == 'Magup':
        if info['currentJob'] != 'Magup':
            info = infocollection.find_one_and_update({},{'$set': {'currentJob': 'Magup'}})
            logString = "Start Magup at " + datetime.now().strftime('%H:%M:%S')
            print(logString)
            logFile.writelines([logString, '\n'])

    elif info['command'] == 'Magdown':
        if info['currentJob'] != 'Magdown':
            info = infocollection.find_one_and_update({},{'$set': {'currentJob': 'Magdown'}})
            logString = "Start Magdown at " + datetime.now().strftime('%H:%M:%S')
            print(logString)
            logFile.writelines([logString, '\n'])

    elif info['command'] == 'Soak':
        if info['currentJob'] != 'Soak':
            info = infocollection.find_one_and_update({},{'$set': {'currentJob': 'Soak'}})
            logString = "Start Soak at " + datetime.now().strftime('%H:%M:%S')
            print(logString)
            logFile.writelines([logString, '\n'])

    else:
        print('No Command')

    # Control Magnet
    job = info['currentJob']

    if job == 'Magup':
        # Increase Magnet Voltage
        print('magup')
    elif job == 'Magdown':
        # Decrease Magnet Voltage
        print('magdown')
    elif job == 'Soak':
        # Soak
        print('soak')
    else:
        print('No Job')

    # Close Log File
    logFile.close()

    # Wait
    time.sleep(5)

magup
magup
magup